main_0에서 데이터 크롤링 및 정제

comment_sentiment_analysis에서 댓글 감성 분석

main_1에서 요약 모델에 들어갈 수 있도록 naver, daum 데이터와 같은 처리 해주기 (stemming, normalizing), model.predict를 할 수 있게끔 convert_token_to_id 해주기

normalizing에서 stopwords_list, least_list 이렇게 두 개를 사용했는데 least_list가 따로 저장되어 있지 않은 것 같아서 stopwords_concat.csv만 사용해 전처리하겠습니다.

In [74]:
import numpy as np
import pandas as pd
import ast

from konlpy.tag import Hannanum
from kobert_transformers import get_tokenizer

In [2]:
channel_a = pd.read_csv('youtube_0816_0822/add_comment_sentiment_data/cs_channel_a_0816_0822.csv')
jtbc = pd.read_csv('youtube_0816_0822/add_comment_sentiment_data/cs_jtbc_0816_0822.csv')
kbs = pd.read_csv('youtube_0816_0822/add_comment_sentiment_data/cs_kbs_0816_0822.csv')
mbc = pd.read_csv('youtube_0816_0822/add_comment_sentiment_data/cs_mbc_0816_0822.csv')
mbn = pd.read_csv('youtube_0816_0822/add_comment_sentiment_data/cs_mbn_0816_0822.csv')
sbs = pd.read_csv('youtube_0816_0822/add_comment_sentiment_data/cs_sbs_0816_0822.csv')
yonhab = pd.read_csv('youtube_0816_0822/add_comment_sentiment_data/cs_yonhab_0816_0822.csv')
ytn = pd.read_csv('youtube_0816_0822/add_comment_sentiment_data/cs_ytn_0816_0822.csv')

In [119]:
channel_a.shape

(71, 20)

In [120]:
jtbc.shape

(188, 21)

In [121]:
kbs.shape

(140, 20)

In [122]:
mbc.shape

(405, 21)

In [123]:
mbn.shape

(165, 20)

In [124]:
sbs.shape

(174, 21)

In [125]:
yonhab.shape

(495, 20)

In [126]:
ytn.shape

(724, 20)

In [17]:
stopwords_df = pd.read_csv('stopwords_concat.csv')
stopwords_list = stopwords_df.stopwords

In [19]:
stopwords_list[:10], stopwords_list[-10:]

(0    이
 1    있
 2    하
 3    것
 4    들
 5    그
 6    되
 7    수
 8    이
 9    보
 Name: stopwords, dtype: object,
 765     둘
 766     셋
 767     넷
 768    다섯
 769    여섯
 770    일곱
 771    여덟
 772    아홉
 773     령
 774     영
 Name: stopwords, dtype: object)

# Stemming and Remove stopwords

+ stemming에서 제거하는 품사

{'E': '어미',
 'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ET': '전성 어미', 
 'S': '기호'}

+ stemming에서 제거하지 않는 품사

{'F': '외국어',
 'I': '독립언',
 'II': '감탄사',
 'J': '관계언',
 'JC': '격조사',
 'JP': '서술격 조사',
 'JX': '보조사',
 'M': '수식언',
 'MA': '부사',
 'MM': '관형사',
 'N': '체언',
 'NB': '의존명사',
 'NC': '보통명사',
 'NN': '수사',
 'NP': '대명사',
 'NQ': '고유명사',
 'P': '용언',
 'PA': '형용사',
 'PV': '동사',
 'PX': '보조 용언',
 'X': '접사',
 'XP': '접두사',
 'XS': '접미사'}

In [41]:
def stemming_and_remove_stopwords(row, tagging_method, stopwords):
    '''기사 본문 품사태깅 후 어미가 아닌 형태소들만 추출한 뒤 불용어 제거'''
    
    stem_list = []
    tagged = tagging_method.pos(row)
    
    for element in tagged:
        # {'E': '어미', 'EC': '연결 어미', 'EF': '종결 어미', 'EP': '선어말어미', 'ET': '전성 어미', 'S': '기호'} 제거
        # 불용어 제거
        if element[1] not in ['E', 'EC', 'EF', 'EP', 'ET', 'S'] and element[0] not in stopwords:
            stem_list.append(element[0])
    
    return stem_list

def df_stemming_and_remove_stopwords(DataFrame, stopwords):
    '''stemming_and_remove_stopwords을 DataFrame에 적용'''
    
    copy = DataFrame.copy()
    
    han = Hannanum() # tagging_method
    cleaned_description = copy.cleaned_description
    stemmed = []
    
    for element in cleaned_description:
        stem = stemming_and_remove_stopwords(row=element,
                                             tagging_method=han,
                                             stopwords=stopwords)
        stemmed.append(stem)
    
    copy['stemmed'] = stemmed
    
    return copy

In [53]:
stemmed_channel_a = df_stemming_and_remove_stopwords(DataFrame=channel_a,
                                                     stopwords=stopwords_list)

In [55]:
stemmed_jtbc = df_stemming_and_remove_stopwords(DataFrame=jtbc,
                                                stopwords=stopwords_list)

In [56]:
stemmed_kbs = df_stemming_and_remove_stopwords(DataFrame=kbs,
                                               stopwords=stopwords_list)

In [57]:
stemmed_mbc = df_stemming_and_remove_stopwords(DataFrame=mbc,
                                               stopwords=stopwords_list)

In [58]:
stemmed_mbn = df_stemming_and_remove_stopwords(DataFrame=mbn,
                                               stopwords=stopwords_list)

In [59]:
stemmed_sbs = df_stemming_and_remove_stopwords(DataFrame=sbs,
                                               stopwords=stopwords_list)

In [60]:
stemmed_yonhab = df_stemming_and_remove_stopwords(DataFrame=yonhab,
                                                  stopwords=stopwords_list)

In [61]:
stemmed_ytn = df_stemming_and_remove_stopwords(DataFrame=ytn,
                                               stopwords=stopwords_list)

pandas에서 DataFrame의 원소로 list가 들어가는데 csv파일로 내렸다가 다시 불러왔을 때 string이 되는거고 그때 string에 ast.literal_eval를 써주면 다시 list로 변환해서 사용 가능

In [76]:
stemmed_ytn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  724 non-null    int64  
 1   Unnamed: 0.1                724 non-null    int64  
 2   published_time              724 non-null    object 
 3   video_id                    724 non-null    object 
 4   video_url                   724 non-null    object 
 5   crawling_time               724 non-null    object 
 6   author                      724 non-null    object 
 7   title                       724 non-null    object 
 8   length                      724 non-null    float64
 9   rating                      724 non-null    float64
 10  views                       724 non-null    float64
 11  thumbnail_url               724 non-null    object 
 12  captions                    724 non-null    object 
 13  description                 724 non

# Convert token to id

In [77]:
def token_to_id_and_padding(DataFrame):
    '''stemmed를 id로 바꾼 뒤 길이에 맞게 padding'''
    
    copy = DataFrame.copy()
    
    # kobert tokenizer 생성
    tokenizer = get_tokenizer()
    pad = tokenizer.pad_token_id

    id_list = []
    for element in copy.stemmed:
        # token을 id로 변환, 최대 길이에 맞게 512번째 token까지만 사용하고 나머진 버림
        element_id = tokenizer.convert_tokens_to_ids(element[:512])
        # 512보다 길이가 짧을 때 pad id인 1로 padding, 모델에 predict할 때 학습할 때 설정한 길이까지만 잘라서 사용
        if len(element_id) < 512:
            element_id = element_id + [pad] * (512 - len(element_id))
        
        id_list.append(element_id)
    
    copy['token_ids'] = id_list
    
    return copy        

In [92]:
id_channel_a = token_to_id_and_padding(stemmed_channel_a)
id_jtbc = token_to_id_and_padding(stemmed_jtbc)
id_kbs = token_to_id_and_padding(stemmed_kbs)
id_mbc = token_to_id_and_padding(stemmed_mbc)
id_mbn = token_to_id_and_padding(stemmed_mbn)
id_sbs = token_to_id_and_padding(stemmed_sbs)
id_yonhab = token_to_id_and_padding(stemmed_yonhab)
id_ytn = token_to_id_and_padding(stemmed_ytn)

In [93]:
id_ytn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  724 non-null    int64  
 1   Unnamed: 0.1                724 non-null    int64  
 2   published_time              724 non-null    object 
 3   video_id                    724 non-null    object 
 4   video_url                   724 non-null    object 
 5   crawling_time               724 non-null    object 
 6   author                      724 non-null    object 
 7   title                       724 non-null    object 
 8   length                      724 non-null    float64
 9   rating                      724 non-null    float64
 10  views                       724 non-null    float64
 11  thumbnail_url               724 non-null    object 
 12  captions                    724 non-null    object 
 13  description                 724 non

In [98]:
print(id_ytn.token_ids[0])

[5585, 0, 5330, 0, 0, 5859, 0, 0, 7905, 7078, 0, 5330, 0, 0, 0, 0, 7078, 5868, 5330, 0, 6116, 7782, 7141, 0, 7086, 6964, 141, 7096, 0, 6116, 0, 6966, 0, 5468, 0, 0, 6116, 6940, 0, 7086, 0, 7171, 7095, 0, 7096, 5886, 6536, 0, 7088, 7404, 7141, 5511, 0, 0, 7782, 0, 5561, 7096, 0, 0, 0, 6916, 7096, 0, 0, 0, 7905, 6903, 0, 0, 7096, 6999, 6265, 7086, 0, 0, 0, 7088, 0, 7782, 6999, 6265, 7086, 6966, 6410, 5804, 5822, 6983, 0, 0, 7095, 0, 6116, 0, 7782, 0, 0, 0, 6903, 0, 0, 6116, 7096, 7096, 0, 0, 6286, 6900, 0, 0, 6903, 0, 0, 6116, 6940, 6999, 0, 6903, 6821, 6629, 7141, 6972, 7078, 0, 5886, 0, 7096, 0, 7096, 6265, 7086, 0, 0, 0, 5468, 234, 7096, 0, 7207, 5666, 5468, 7207, 6412, 7337, 7088, 7404, 0, 6116, 0, 7782, 0, 7096, 7096, 7337, 0, 6534, 7096, 0, 7782, 0, 7088, 0, 0, 7788, 0, 7171, 0, 6896, 0, 7782, 6999, 0, 7171, 7096, 0, 0, 7295, 7096, 5850, 7096, 0, 5592, 0, 0, 6116, 6374, 0, 7096, 6307, 7078, 0, 0, 7088, 7140, 5330, 5377, 0, 7095, 0, 7088, 7404, 5398, 7096, 6150, 0, 0, 6896, 0, 7782,

# 필요없는 부분 제거

In [102]:
id_channel_a.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [103]:
id_jtbc.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [104]:
id_kbs.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [105]:
id_mbc.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [106]:
id_mbn.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [107]:
id_sbs.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [108]:
id_yonhab.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [100]:
id_ytn.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [109]:
id_jtbc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              188 non-null    object 
 1   video_id                    188 non-null    object 
 2   video_url                   188 non-null    object 
 3   crawling_time               188 non-null    object 
 4   author                      188 non-null    object 
 5   title                       188 non-null    object 
 6   length                      188 non-null    float64
 7   rating                      188 non-null    float64
 8   views                       188 non-null    float64
 9   thumbnail_url               188 non-null    object 
 10  captions                    188 non-null    object 
 11  description                 188 non-null    object 
 12  comments                    188 non-null    object 
 13  article_url                 188 non

In [110]:
id_ytn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              724 non-null    object 
 1   video_id                    724 non-null    object 
 2   video_url                   724 non-null    object 
 3   crawling_time               724 non-null    object 
 4   author                      724 non-null    object 
 5   title                       724 non-null    object 
 6   length                      724 non-null    float64
 7   rating                      724 non-null    float64
 8   views                       724 non-null    float64
 9   thumbnail_url               724 non-null    object 
 10  captions                    724 non-null    object 
 11  description                 724 non-null    object 
 12  comments                    724 non-null    object 
 13  cleaned_description         724 non

In [117]:
id_channel_a.to_csv('youtube_0816_0822/convert_token_to_id/id_channel_a_0816_0822.csv')
id_jtbc.to_csv('youtube_0816_0822/convert_token_to_id/id_jtbc_0816_0822.csv')
id_kbs.to_csv('youtube_0816_0822/convert_token_to_id/id_kbs_0816_0822.csv')
id_mbc.to_csv('youtube_0816_0822/convert_token_to_id/id_mbc_0816_0822.csv')
id_mbn.to_csv('youtube_0816_0822/convert_token_to_id/id_mbn_0816_0822.csv')
id_sbs.to_csv('youtube_0816_0822/convert_token_to_id/id_sbs_0816_0822.csv')
id_yonhab.to_csv('youtube_0816_0822/convert_token_to_id/id_yonhab_0816_0822.csv')
id_ytn.to_csv('youtube_0816_0822/convert_token_to_id/id_ytn_0816_0822.csv')

#불러올 때 index_col=0 옵션 사용

#a = pd.read_csv('youtube_0816_0822/convert_token_to_id/id_channel_a_0816_0822.csv', index_col=0)

DataFrame 불러온 다음에 token_ids 열을 model.predict에 사용한 뒤 tokenizer.convert_ids_to_tokens()으로 다시 token으로 변환